In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow_hub as thub
import tensorflow.keras.backend as K

from lib.data_utils import *

In [2]:
from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets
import art

In [3]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [4]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input
# from tensorflow.keras import Sequential
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.losses import CategoricalCrossentropy
# from tensorflow.keras.utils import to_categorical


In [5]:
# Load the raw CIFAR-10 data
cifar10_dir = 'lib/datasets/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(cifar10_dir)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [63]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [47]:
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

ganeval_module = thub.Module(model_url)
# gan_eval_layer = thub.KerasLayer(model_url)

In [48]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [49]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [64]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


### Testing the retrieved hub model by evaluating on train data
Should give us 100% or something close to that

In [50]:
ge_cifar_clf.evaluate(testd, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0017 - acc: 1.0000


[0.001668029203079641, 1.0]

### Testing a sample attack

In [51]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.4)

In [52]:
x_test_adv = attack_fgsm.generate(testd.copy())

In [53]:
ge_cifar_clf.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 6.9334 - acc: 0.1020


[6.933382461547851, 0.102]

### Define the substitute classifier for our model

In [14]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [65]:
def build_substitute_model():
    model = tf.keras.Sequential( )
    model.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
    model.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
    model.add( MaxPooling2D( pool_size=(2, 2) ) )
    model.add( Flatten( ) )
    model.add( Dense( 128, activation='relu' ) )
    model.add( Dense(10, activation='softmax' ) )
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=[ 'accuracy' ] )
    
    return model

In [55]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [56]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=15)

50000 7500
(7500, 32, 32, 3) (7500, 10)


In [57]:
cloud_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
extracted_art_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=False)

In [58]:
attack_CCC = CopycatCNN(classifier=cloud_clf, 
                        batch_size_fit=8, 
                        batch_size_query=8,
                        nb_epochs=20,
                        nb_stolen=1000)

In [59]:
attack_CCC.extract(partial_x_train, partial_y_train, thieved_classifier=extracted_art_clf)

Epoch 1/20
125/125 [==============================] - 3s 21ms/step - loss: 2.1634 - acc: 0.2060
Epoch 2/20
125/125 [==============================] - 2s 15ms/step - loss: 1.5234 - acc: 0.4780
Epoch 3/20
125/125 [==============================] - 2s 17ms/step - loss: 1.0434 - acc: 0.6430
Epoch 4/20
125/125 [==============================] - 2s 18ms/step - loss: 0.7134 - acc: 0.7800
Epoch 5/20
125/125 [==============================] - 2s 18ms/step - loss: 0.4941 - acc: 0.8480
Epoch 6/20
125/125 [==============================] - 2s 18ms/step - loss: 0.3220 - acc: 0.9010
Epoch 7/20
125/125 [==============================] - 2s 18ms/step - loss: 0.1819 - acc: 0.9460
Epoch 8/20
125/125 [==============================] - 2s 17ms/step - loss: 0.1265 - acc: 0.9570
Epoch 9/20
125/125 [==============================] - 2s 17ms/step - loss: 0.0724 - acc: 0.9820
Epoch 10/20
125/125 [==============================] - 2s 18ms/step - loss: 0.0568 - acc: 0.9850
Epoch 11/20
125/125 [==================

art.classifiers.keras.KerasClassifier(model=<tensorflow.python.keras.engine.sequential.Sequential object at 0x14f4f1518>, use_logits=False, channel_index=3, clip_values=(0, 1), preprocessing_defences=None, postprocessing_defences=None, preprocessing=(0, 1), input_layer=0, output_layer=0)

In [36]:
art_clf._model.evaluate(x_train_adv, y_train_adv)
extracted_art_clf._model.evaluate(x_train_adv, y_train_adv)

1000/1000 [==============================] - 0s 258us/sample - loss: 4.6095 - acc: 0.4210


[4.609531337738037, 0.421]

In [37]:
art_clf._model.evaluate(x_test_adv, y_test_adv)
extracted_art_clf._model.evaluate(x_test_adv, y_test_adv)

1000/1000 [==============================] - 0s 255us/sample - loss: 4.5158 - acc: 0.4180


[4.515756118774414, 0.418]